# Augmenting a dataset for instance segmentation

In this notebook, we illustrate how CLODSA can be employed to augment a dataset of images devoted to instance segmentation that was annotated using the [COCO format](http://cocodataset.org/#home). 

Ensure that there's an input folder and that it contains the following:
- the images we want to augment (jpeg/jpg, png and webp only)
- a single json file in COCO format that contains all annotations for the added images, and its name is **annotations.json**

In [ ]:
!ls input | wc -l

We create the output folder.

In [ ]:
!mkdir output

## Augmentation techniques

For this example, we consider three augmentation techniques. 

The augmentation techniques applied in this example are:
- Rotation
- Flip
- Dropout
- Elastic Deformation
- Equalize Histogram
- Gamma Correction
- Gaussian Blurring
- Gaussian Noise
- Median Blurring
- Raise Blue Channel
- Raise Green Channel
- Raise Hue
- Salt and Pepper


## Installing the necessary libraries

In case that CLODSA is not installed in your system, the first task consists in installing it using ``pip``.

In [ ]:
!pip install clodsa

## Loading the necessary libraries

The first step in the pipeline consists in loading the necessary libraries to apply the data augmentation techniques in CLODSA.

In [ ]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
import cv2
%matplotlib inline

## Creating the augmentor object

As explained in the documentation of CLODSA, we need to specify some parameters for the augmentation process, and use them to create an augmentor object.  

_The kind of problem_. In this case, we are working in an instance segmentation problem.

In [ ]:
PROBLEM = "instance_segmentation"

_The annotation mode_. The annotation is provided using the coco format in a file called annotations.json. 

In [ ]:
ANNOTATION_MODE = "coco"

_The input path_. The input path containing the images. 

In [ ]:
INPUT_PATH = "input"

_The generation mode_. In this case, linear, that is, all the augmentation techniques are applied to all the images of the original dataset. 

In [ ]:
GENERATION_MODE = "linear"

_The output mode_. The generated images will be stored in a new folder called output.  

In [ ]:
OUTPUT_MODE = "coco"
OUTPUT_PATH= "output/"

Using the above information, we can create our augmentor object. 

In [ ]:
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH})

## Adding the augmentation techniques

Now, we define the techniques that will be applied in our augmentation process and add them to our augmentor object. To illustrate the transformations, we will use the following image of the dataset. 



First of all, we must define a transformer generator.

In [ ]:
transformer = transformerGenerator(PROBLEM)

_Rotation:_

In [ ]:
for angle in [90,180]:
    rotate = createTechnique("rotate", {"angle" : angle})
    augmentor.addTransformer(transformer(rotate))

_Flip:_

In [ ]:
flip = createTechnique("flip",{"flip":1})
augmentor.addTransformer(transformer(flip))

*Dropout:*

In [ ]:
dropout = createTechnique("dropout",{"percentage":0.05})
augmentor.addTransformer(transformer(dropout))

*Elastic Deformation:*

In [ ]:
elastic = createTechnique("elastic",{"alpha":5,"sigma":0.05})
augmentor.addTransformer(transformer(elastic))

*Equalize Histogram:*

In [ ]:
histograms = createTechnique("equalize_histogram",{})
augmentor.addTransformer(transformer(histograms))

*Gamma Correction:*

In [ ]:
gamma = createTechnique("gamma",{"gamma":1.5})
augmentor.addTransformer(transformer(gamma))

*Gaussian Blurring:*

In [ ]:
gaussian_blurring = createTechnique("gaussian_blur", {"kernel" : 5})
augmentor.addTransformer(transformer(gaussian_blurring))

*Gaussian Noise:*

In [ ]:
gaussian_noise = createTechnique("gaussian_noise", {"mean" : 0,"sigma":10})
augmentor.addTransformer(transformer(gaussian_noise))

*Raise Hue:*

In [ ]:
raise_hue = createTechnique("raise_hue", {"power" : 0.9})
augmentor.addTransformer(transformer(raise_hue))

*Salt and Pepper:*

In [ ]:
salt_and_pepper = createTechnique("salt_and_pepper", {"low" : 0,"up":25})
augmentor.addTransformer(transformer(salt_and_pepper))

## Applying the augmentation process

Finally, we apply the augmentation process (this might take some time depending on the number of images of the original dataset and the number of transformations that will be applied). 

Warning: in case of info error, edit annotations.json and add the following:


```
"info": {
    "description": "Example Dataset",
    "version": "0.1.0",
    "year": 2022,
    "contributor": "your name"
  },
  "licenses": [
    {
      "id": 1,
      "name": "",
      "url": ""
    }
  ],
```



In [ ]:
augmentor.applyAugmentation()

We can now check the amount of files of the output folder (includes both images and the annotations.json)

In [ ]:
!ls output/ | wc -l

1387


Finally, we can visualize the results using some of the tools provided by [the COCO API](https://github.com/cocodataset/cocoapi).

In [ ]:
!pip install pycocotools

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

In [ ]:
image_directory = 'output/'
annotation_file = 'output/annotation.json'

In [ ]:
example_coco = COCO(annotation_file)

In [ ]:
categories = example_coco.loadCats(example_coco.getCatIds())
category_names = [category['name'] for category in categories]
print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))

category_names = set([category['supercategory'] for category in categories])
print('Custom COCO supercategories: \n{}'.format(' '.join(category_names)))

The following creates a zip with the contents of the output file:

In [ ]:
!zip -r output.zip output